In [7]:
import gym # reinforcement learning için env sağlar, geliştirme ortamı sağlar
import numpy as np
from collections import deque # ajanın belleğini tanımlamak için gerekli deque veri yapısı
from tensorflow.keras.models import Sequential # sıralı model oluşturmak için
from tensorflow.keras.layers import Dense # tam bağlantılı (dense) katmanlar
from tensorflow.keras.optimizers import Adam

import random

from tqdm import tqdm # ilerlemeyi görselleştimek için


In [9]:
# dql agent calss

class DQLAgent:
    def __init__(self, env):# parametreleri ve hiperparametreleri tanımla env:cartpole

      #cevrinin gözlem alanı (state) boyut sayısı
      self.state_size = env.observation_space.shape[0]

      # cevrede bulunan eylem sayısı (ajanın secebileceği eylem sayısı)
      self.action_size = env.action_space.n

      # gelecekteki ödüllerin indirim oranı
      self.gamma = 0.95

      # learning rate ajanın öğrenme hızı
      self.learning_rate = 0.001

      # kesfetme oranı (epsilon) epsilon = 1 olsun maximum kesif
      self.epsilon = 1

      # epsilonun her iterasyonda azalma oranı (epsilon azaldıkça daha fazla öğrenme, daha az keşif)
      self.epsilon_decay = 0.995

      # minimum keşfetme oranı (epsilon 0.01'in altına inemez)
      self.epsilon_min = 0.01

      # ajanın deneyimleri = bellek = geçmiş adımlar
      self.memory = deque(maxlen=1000)

      # derin öğrenme modelini inşaa et
      self.model = self.build_model()


    def build_model(self): #deep q learning sinir ağı modelini oluştur

      model = Sequential()# sıralı model

      # girdi katmanı, 48 nöron, relu activation
      model.add(Dense(48, input_dim=self.state_size, activation='relu'))

      # 2. gizli katman, 48 nöron
      model.add(Dense(48, activation='relu'))

      #output katmanı
      model.add(Dense(self.action_size, activation='linear'))

      # modelin derlenmesi
      model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))

      return model

    def remember(self,state,action,reward,next_state,done): # ajanın deneyimlerini bellek veri yapısına kaydet

      self.memory.append((state,action,reward,next_state,done))

    def act(self,state): # ajanımız eylem secebilecek

      # eğer rastgele üretilen sayı epsilondan küçükse rastgele eylem seçilir (keşif)
      if random.uniform(0,1) <= self.epsilon:
        return env.action_space.samples()# rastgele eylem seç

        # aksi dürümda model tarafından tahmin edilen değerleri göre en iyi eylem seçilir
        act_values = self.model.predict(state,velbose=0)

        # en yüksek değere sahip eylemi seç
        return np.argmax(act_values[0])

    def replay(self,batch_size):#deneyimleri tekrar oynatarak deep q ağı eğitilir

      # bellekte yeterince deneyim yoksa geri oynatma yapılamaz
      if len(self.memory) < batch_size:
        return

        # rastgele deneyimleri alın
        minibatch = random.sample(self.memory,batch_size)

        for state,action,reward,next_state,done in minibatch:
          if done: # eğer done ise bitiş durumu var ise, ödülü doğrudan hedef olarak alırız
            target = reward

          else:
            target = reward + self.gamma * np.amax(self.model.predict(next_state,velbose=0)[0])

          #modelin tahmin ettiği ödüller
          train_target = self.model.predict(state,verbose=0)

          #ajanın yaptığı eyleme göre tahmin edilen ödülü gencelle
          train_target[0][action] = target

          #modeli eğit
          self.model.fit(state,train_target,verbose=0)



    def adaptiveEGreedy(self,): # epsilonun zamanla azalması yani kesif ve sömürü dengesi

      if self.epsilon > self.epsilon_min:
        self.epsilon = self.epsilon * self.epsilon_decay





In [8]:
env = gym.make('CartPole-v1',render_mode='human') # cartpole ortamını başlatıyoruz
agent = DQLAgent(env)

batch_size = 32 # eğitim için minibatch boyutu
episodes = 1000 # epoch, simulasyonun oynatılacağı toplam bölüm sayısı

/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
for e in tqdm(range(episodes)):

  # ortamı sıfırla başlangıç durumunu al
  state = env.reset()[0]# ortamı sıfırlamak
  state = np.reshape(state, [1,4])

  time = 0 # zaman adımını başlat

  while True:

    # ajanın eylem seçer
    action = agent.act(state)

    # ajanımız ortamda bu eylemi uygular ve bu eylem sonucunda next_state,reward,bitiş bilgisi (done) alır
    (next_state,reward,done,_ ,_) = env.step(action)
    next_state = np.reshape(next_state, [1,4])

    # yapmış olduğu bu adımı yani eylemi ve bu eylem sonucu env'dan alınan bilgileri kaydeder
    agent.ramember(state,action,reward,next_state,done)

    # mevcut durumu günceller
    state = next_state

    #deneyimlerden eniden oynatmayı başlatır reply() -> training
    agent.adaptiveEGreedy()

    # zaman adımını arttırır
    time = time+1

    # eğer done ise döngüyü kırır ve bölüm biter ve yeni bölüme başlar
    if done:
      print(f"Episode: {e}, time: {time}")
      break

  0%|          | 0/1000 [00:00<?, ?it/s]


ValueError: cannot reshape array of size 1 into shape (1,4)

In [ ]:
# test
import time

trained_model = agent # eğitilen modeli al

env = gym.make("CartPole-v1",render_mode="human") # cartpole ortamını bastırıyoruz
state = env.reset()[0]
state = np.reshape(state, [1,4])

time_t = 0 # zaman adımını başlat

while True:
  env.render() #ortamın görsel olarak render et
  action = trained_model.act(state)# eğitilen modelden action gerçekleştir
  (next_state,reward,done,_ ,_) = env.step(action) # eylemi uygula
  next_state = np.reshape(state, [1,4])
  time_t = time_t + 1 # durumu güncelle
  time_t +=1
  print(f"Time: {time_t}")

  time.sleep(0.5)# yarım saniye bekle

  if done:
    break

print("Done")